<a href="https://colab.research.google.com/github/Bunto4/scrapyscrapyscrapypy/blob/main/Imdb_250.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scrapy

In [ ]:
import scrapy
from scrapy.http import HtmlResponse
from scrapy import Selector
import requests

In [ ]:
class ImdbSpider(scrapy.Spider):
  name = "imdb"
  allowed_domains = ["imdb.com"]
  start_urls = ['http://www.imdb.com/chart/top']

  def parse (self, response):
    # nonweeme mad-mus co crpoxam, xapaxrepusynww Ton-(prams:
    table_rows = response.xpath(
    './/*[@class="chart full-width" and @data-caller-name="chart-top250movie"]/'
    'tbody[@class="lister-list"]/tr'
    )

    for row in table_rows:
      yield {"title" : row.xpath("./td[@class='titleColumn']/a/text() ").extract_first(),
               "year": row.xpath("./td[@class='titleColumn']/span/text()").extract_first().strip("() "),
               "rating": row.xpath("./td[@class='ratingColumn imdbRating']/strong/text()").extract_first(),
               }

In [ ]:
class MovieItem(scrapy.Item):
  title = scrapy.Field()
  rating = scrapy.Field()
  summary = scrapy.Field()
  year = scrapy.Field()
  genre = scrapy.Field()
  runtime = scrapy.Field()
  directors = scrapy.Field()
  writers = scrapy.Field()
  cast = scrapy.Field()

class ActorItem(scrapy.Item):
  actor_name = scrapy.Field()
  character = scrapy.Field()

In [ ]:
class ImdbSpider_2(scrapy.Spider):
  name = "imdb"
  allowed_domains = ["imdb.com"]
  base_url = 'https://imdb.com'
  start_urls = ['http://www.imdb.com/chart/top']

  def parse(self,response):
    table_rows = response.xpath(
        './/*[@class="chart full-width" and @data-caller-name="chart-top250movie"]/'
    'tbody[@class="lister-list"]/tr'
    )

    for row in table_rows:
      rating = row.xpath("./td[@class='ratingColumn imdbRating']/strong/text()").extract_first()
      ret_url = row.xpath("td[@class='titleColumn']/a/@href").extract_first().strip()
      row_url = self.base_url + ret_url
      yield scrapy.Request(row_url,callback = self.parseOneMovie,meta={'rating':rating})

  def parseOneMovie(self,response):
    item = MovieItem()

    item['rating'] = response.meta['rating'][0]
    item['title'] = response.xpath('.//*/div[@class="sc-80d4314-1 fbQftq"]/h1/text()').extract_first().strip()
    #item['title'] = response.xpath(".//*/div[@class='title_wrapper']/h1/text()").extract_first().strip()
    item['year'] = response.xpath('//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/div/ul/li[1]/span/text()').extract()[0]
    


    #item['summary'] = response.xpath(".//*/div[@class='summary_text']/text()").extract_first().strip()
    #item['directors'] = response.xpath('.//*/div[@class="credit_summary_item"]/a/text()').extract_first.strip()
    #item['writers'] = response.xpath('.//*/div[@class="credit_summary_item"]/a/text()').extract()[1].strip()

    #item['cast'] = list()

    #for cast in response.xpath(".//table[@class='cast_list]/tr")[1:]:
      #actor = ActorItem()

      #actor['actor_name']= cast.xpath("./td[2]/a/text()").extract_first().strip()
      #actor['character'] = cast.xpath("./td[@class='character']/a/text()").extract_first()

      #item['cast'].append(actor)

    return item




In [ ]:
DEBUG_URL = 'http://www.imdb.com/chart/top'

req = requests.get(DEBUG_URL)

response = HtmlResponse(url=DEBUG_URL, body=req.content)
selector = Selector(response=response)

In [ ]:
response.body

In [ ]:
DEBUG_URL = 'https://www.imdb.com/title/tt0111161'

req = requests.get(DEBUG_URL)

response = HtmlResponse(url=DEBUG_URL, body=req.content)
selector = Selector(response=response)



In [ ]:
e = selector.xpath (
    '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/div/ul/li[1]/span/text()'
).extract()[0]
int(e)

In [ ]:
from scrapy.crawler import CrawlerProcess

process = CrawlerProcess()
process.crawl(ImdbSpider_2)
process.start()

In [ ]:
from scrapy.crawler import CrawlerProcess
process = CrawlerProcess(settings={
    "FEEDS": {
        "Ruslan.csv": {"format": "csv"},
    },
})

process.crawl(ImdbSpider_2)
process.start()


In [ ]:
from scrapy.spiders import crawl


In [ ]:
crawl.Request

In [ ]:
import pandas as pd

data = pd.read_csv('/content/Ruslan.csv')

In [ ]:
data